In [5]:
import os
import sys
jupyter_module_path = os.path.join( os.getcwd(), '..', '..' )
sys.path.append( jupyter_module_path )
jupyter_module_path = os.path.join( os.getcwd(), '..', '..', 'EfJupyter' )
sys.path.append( jupyter_module_path )

from EfJupyter import EfConf

ImportError: cannot import name 'ExternalFieldElectricOnRegularGridFromH5File'

In this example, we compare an exact analytical trajectory of a charged particle moving in a uniform magnetic field with results of a numerical simulation.

A charged particle moves in uniform magnetic field in circular trajectory. 
An exact expressions for the trajectory can be obtained by solving Newton's equation:

$$
m \frac{ d \textbf{v} }{ dt } = \frac{q}{c} [ \textbf{v} \ \textbf{H} ]
$$

We chose z-axis to be directed along the magnetic field. 
Then the previous expression becomes:

\begin{align}
& m v_x' = \frac{q}{c} v_y H_z \\
& m v_y' = -\frac{q}{c} v_x H_z \\
& m v_z' = 0
\end{align}

Performing integration over time (see [Single Particle in Uniform Magnetic Field](https://github.com/epicf/ef/blob/dev/examples/single_particle_in_magnetic_field/Single%20Particle%20in%20Uniform%20Magnetic%20Field.ipynb) [SymPy](http://www.sympy.org/en/index.html) notebook), it is possible to obtain
particle's law of motion:

\begin{align}
&v_x = v_{x0} \cos( \Omega t ) + v_{y0} \sin( \Omega t )
\\
&v_y = -v_{x0} \sin( \Omega t ) + v_{y0} \cos( \Omega t )
\\
&v_z = v_{z0}
\end{align}

\begin{align}
&x = x_0 + \frac{1}{\Omega} \Big[ v_{x0} \sin( \Omega t ) - v_{y0} \cos( \Omega t ) + v_{y0} \Big]
\\
&y = y_0 + \frac{1}{\Omega} \Big[ v_{x0} \cos( \Omega t ) + v_{y0} \sin( \Omega t ) - v_{x0} \Big]
\\
&z = z_0 + v_{z0} t
\end{align}


The parameters of the [circular motion](https://en.wikipedia.org/wiki/Gyroradius) - cyclotron frequency $\Omega$ and Larmor radius $r$ - are

$$
\Omega = \frac{q H_z}{m c},
\qquad
r = \frac{m c v_{\perp}}{q H_z} = \frac{m c (v_x^2 + v_y^2)^{1/2}}{q H_z}
$$


Let's perform some estimates (see below).
Suppose there is an electron (`q = 4.8e-10 [cgs]`, `m = 9.1e-28 [g]`) moving 
in the `1000 [Gs] = 1 * 10^-1 [T]` magnetic field. It's speed along the field (z-axis) corresponds
to 1 keV energy, and it's speed in perpendicular plane corresponds to energy 100 eV (initially distributed
evenly between x- and y-components).

In [9]:
from math import *

m = 9.8e-28
q = 4.8e-10
print( "q = {:.3e} [cgs]".format( q ) )
print( "m = {:.3e} [g]".format( m ) )

ev_to_cgs = 1.60218e-12
E_along = 1000 * ev_to_cgs
v_along = sqrt( 2 * E_along / m )
E_perp = 100 * ev_to_cgs
v_perp = sqrt( 2 * E_perp/2 / m )
print( "E_along = {:.3e} [eV] = {:.3e} [erg]".format( E_along / ev_to_cgs, E_along ) )
print( "E_perp = {:.3e} [eV] = {:.3e} [erg]".format( E_perp / ev_to_cgs, E_perp ) )
print( "v_along = {:.3e} [cm/s]; p_along = {:.3e} [g * cm/s]".format( v_along, v_along * m ) )
print( "v_perp = {:.3e} [cm/s]; p_perp = {:.3e} [g * cm/s]".format( v_perp, v_perp * m ) )

q = 4.800e-10 [cgs]
m = 9.800e-28 [g]
E_along = 1.000e+03 [eV] = 1.602e-09 [erg]
E_perp = 1.000e+02 [eV] = 1.602e-10 [erg]
v_along = 1.808e+09 [cm/s]; p_along = 1.772e-18 [g * cm/s]
v_perp = 4.043e+08 [cm/s]; p_perp = 3.962e-19 [g * cm/s]


It's cyclotron frequency and Larmor radius are `Omega = 1.633e+10 [1/s], r = 3.502e-02 [cm] ~ 0.035 [mm]`.
Gyration period equals to `3.848e-10 [s]` and distance in the direction of the field covered 
during single revolution is `6.959e-01 [cm] ~ 7 [mm]`

In [10]:
H = 1000
speed_of_light = 3.0e10
cyclotron_fr = q * H / m / speed_of_light
cyclotron_period = 2.0 * pi / cyclotron_fr
single_period_distance_along_field = v_along * cyclotron_period
larmor_r = m * speed_of_light * sqrt(2 * E_perp / m) / q / H
print( "H = {:.3e} [Gs]".format( H ) )
print( "c = {:.3e} [cm/s]".format( speed_of_light ) )
print( "Omega = {:.3e} [1/s]".format( cyclotron_fr ) )
print( "Cyclotron period = {:.3e} [s]".format( cyclotron_period ) )
print( "Single period distance along field= {:.3e} [cm]".format( single_period_distance_along_field ) )
print( "Larmor_r = {:.3e} [cm]".format( larmor_r ) )

H = 1.000e+03 [Gs]
c = 3.000e+10 [cm/s]
Omega = 1.633e+10 [1/s]
Cyclotron period = 3.848e-10 [s]
Single period distance along field= 6.959e-01 [cm]
Larmor_r = 3.502e-02 [cm]


Now, to check this, in config file we define a source which
generates a single particle at startup. Config is mostly similar
to ['Singe Particle in Free Space'](https://github.com/epicf/ef/wiki/Single-Particle-In-Free-Space) example, except this time we need nonzero magnetic field:

In [6]:
single_particle_in_magnetic_field = EfConf()

single_particle_in_magnetic_field.add_field( ExternalFieldMagneticUniform(
    magnetic_field_x = 0,
    magnetic_field_y = 0,
    magnetic_field_z = 1000 ))


NameError: name 'EfConf' is not defined

Total simulation time has been adjusted to approximately 10 full gyration cycles (more precisely, 3.0e-9/3.848e-10 = 7.8 revolutions ).

In [11]:
ten_revolutions_time = 10 * cyclotron_period
print( "10 revolutions time = {:.3e} [s]".format( ten_revolutions_time ) )

sim_time = 3.0e-9
n_of_revolutions = sim_time / cyclotron_period
n_of_steps = 1000
dt = sim_time / n_of_steps
print( "simulation_time = {:.3e} [s]".format( sim_time ) )
print( "n_of_revolutions = {:.1f}".format( n_of_revolutions ) )
print( "number_of_time_steps = {:d}".format( n_of_steps ) )
print( "time_step_size = {:.3e} [s]".format( dt ) )

10 revolutions time = 3.848e-09 [s]
simulation_time = 3.000e-09 [s]
n_of_revolutions = 7.8
number_of_time_steps = 1000
time_step_size = 3.000e-12 [s]


In [ ]:
single_particle_in_magnetic_field.time_grid = TimeGrid(
    total_time = 3.0e-9,
    time_step_size = 3.0e-12,
    time_save_step = 3.0e-12 )

Distance covered is

In [ ]:
z_distance = 5
t = z_distance / v_along
print( "z_distance = {:f} [cm]".format( z_distance ) )
print( "t = {:.3e} [s]".format( t ) )


_construct remaining fields and run_

Processing of simulation results is also mostly similar to the [previous example](https://github.com/epicf/ef/wiki/Single-Particle-in-Free-Space).
The law of motion is more complicated this time, and to define it we need not only
particle mass and initial position and momentum, but also it's charge and magnitude of the magnetic field.
All of this can be extracted from the relevant sections of the output `*.h5` files. 
(see functions `eval_an_trajectory_at_num_time_points`, `velocities` and `coords` at 
[plot.py](https://github.com/epicf/ef/blob/dev/examples/single_particle_in_magnetic_field/plot.py))

_functions_

Trajectory plots:

_figures_

Apart from the trajectory, kinetic energy is also plotted. 
In theory, particle should not gain or loss any energy by interacting with magnetic field. 
However, [due to numerical errors](https://en.wikipedia.org/wiki/Energy_drift), this might not be the case in simulations. In this case, the energy is conserved.

_figures_

**Long Simulation Time**  
[Boris](https://en.wikipedia.org/wiki/Particle-in-cell#The_particle_mover) integration scheme is [notable](http://aip.scitation.org/doi/abs/10.1063/1.4818428?journalCode=php) for energy conservation even for long simulation times. 

_config and results_

**Time Step Larger Than Gyration Period**  
In practice it is necessary to keep in mind that time step should not exceed gyration period. 
Otherwise, this will lead to erroneous results.

_config and results_